In [86]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [87]:
with open("../Makemore/names.txt") as f:
    names = f.read().splitlines()

In [88]:
len(names)

32033

In [89]:
names[:2]

['emma', 'olivia']

In [90]:
char_set = sorted(list({c for name in names for c in name}))
char_set.insert(0, '.')

In [91]:
stoi = {ch: pos for pos, ch in enumerate(char_set)}
itos = {v: k for k, v in stoi.items()}

In [92]:
block_size = 3
X, Y = [], []
for name in names:
    name = '.' * block_size + name + '.'
    for i in range(len(name) - block_size):
        X.append([stoi[ch] for ch in name[i: i+block_size]])
        Y.append(stoi[name[i+block_size]])
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [93]:
train_idx, val_idx, test_idx = torch.utils.data.random_split(range(X.shape[0]), [0.8, 0.1, 0.1])
X_train, Y_train = X[train_idx], Y[train_idx]
X_val, Y_val = X[val_idx], Y[val_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

In [94]:
len(train_idx), len(val_idx), len(test_idx)

(182517, 22815, 22814)

In [ ]:
#pytorchified code

In [115]:
gen = torch.Generator().manual_seed(2147483647)

class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=gen) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
            
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
        
    def parameters(self):
        return [self.gamma, self.beta]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []
        

In [116]:
n_embd = 10
n_hidden = 100
vocab_size = len(itos)
block_size = 3

C = torch.randn(vocab_size, n_embd)
layers = [
    Linear(n_embd * block_size , n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size)
]

with torch.no_grad():
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
              layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

47024


In [117]:
batch_size = 64

for i in range(200000):
    ix = torch.randint(0, X_train.shape[0], (batch_size,), generator = gen)
    Xb, Yb = X_train[ix], Y_train[ix] # creating a barch from X_train
    
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for n, layer in enumerate(layers):
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    for layer in layers:
        layer.out.retain_grad()
    
    for p in parameters:
        p.grad = None
    
    loss.backward()
    
    lr = 0.1 if i < 100000 else 0.01
    
    for p in parameters:
        p.data += -lr * p.grad
    
    if i % 10000 == 0:
        print(f"Loss: {loss.item(): .4f}")

Loss:  3.3128
Loss:  2.3947
Loss:  2.0023
Loss:  2.2459
Loss:  1.8606
Loss:  1.9635
Loss:  2.0769
Loss:  2.0501
Loss:  2.3948
Loss:  2.1690
Loss:  1.7648
Loss:  2.1733
Loss:  1.9328
Loss:  2.0628
Loss:  2.1502
Loss:  2.5297
Loss:  1.9693
Loss:  2.1431
Loss:  2.3175
Loss:  1.9409


In [84]:
@torch.no_grad()
def check_loss(split='train'):
    x, y = {'train': (X[train_idx], Y[train_idx]), 'val': (X[val_idx], Y[val_idx]), 'test': (X[test_idx], Y[test_idx])}[split]
    emb = C[x]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, y)
    print(f"{split.capitalize()} loss: {loss.item()}")

In [85]:
check_loss('train')
check_loss('val')
check_loss('test')

Train loss: 2.0057625770568848
Val loss: 2.0901241302490234
Test loss: 2.0957562923431396
